In [12]:
from tensorflow import keras
import re
# Importing our translations
data_path = "untitled.txt"
# Defining lines as a list of each line
with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')

# Building empty lists to hold sentences
input_docs = []
target_docs = []
# Building empty vocabulary sets
input_tokens = set()
target_tokens = set()

for line in lines:
  # Input and target sentences are separated by tabs
    input_doc, target_doc = line.split('\t')
  # Appending each input sentence to input_docs
    input_docs.append(input_doc)
  # Splitting words from punctuation
    target_doc = " ".join(re.findall(r"[\w']+|[^\s\w]", target_doc))
  # Redefine target_doc below and append it to target_docs:
    target_doc = '<START> ' + target_doc + ' <END>'
    target_docs.append(target_doc)
  
  # Now we split up each sentence into words and add each unique word to our vocabulary set
    for token in re.findall(r"[\w']+|[^\s\w]", input_doc):
        print(token)

        if token not in input_tokens:
            input_tokens.add(token)
    for token in target_doc.split():
        print(token)

        if token not in target_tokens:
            target_tokens.add(token)

input_tokens = sorted(list(input_tokens))
target_tokens = sorted(list(target_tokens))

# Create num_encoder_tokens and num_decoder_tokens:
num_encoder_tokens = len(input_tokens)
num_decoder_tokens = len(target_tokens)

max_encoder_seq_length = max([len(re.findall(r"[\w']+|[^\s\w]", input_doc)) for input_doc in input_docs])
max_decoder_seq_length = max([len(re.findall(r"[\w']+|[^\s\w]", target_doc)) for target_doc in target_docs])

We'll
see
.
<START>
Después
veremos
.
<END>
We'll
see
.
<START>
Ya
veremos
.
<END>
We'll
try
.
<START>
Lo
intentaremos
.
<END>
We've
won
!
<START>
¡
Hemos
ganado
!
<END>
Well
done
.
<START>
Bien
hecho
.
<END>
What's
up
?
<START>
¿
Qué
hay
?
<END>
Who
cares
?
<START>
¿
A
quién
le
importa
?
<END>
Who
drove
?
<START>
¿
Quién
condujo
?
<END>
Who
drove
?
<START>
¿
Quién
conducía
?
<END>
Who
is
he
?
<START>
¿
Quién
es
él
?
<END>
Who
is
it
?
<START>
¿
Quién
es
?
<END>


In [13]:
from tensorflow import keras
from keras.layers import Input, LSTM, Dense
from keras.models import Model

# Encoder training setup
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder_lstm = LSTM(256, return_state=True)
encoder_outputs, state_hidden, state_cell = encoder_lstm(encoder_inputs)
encoder_states = [state_hidden, state_cell]

# Decoder training setup:
decoder_inputs = Input(shape=(None, num_decoder_tokens))
decoder_lstm = LSTM(256, return_sequences=True, return_state=True)
decoder_outputs, decoder_state_hidden, decoder_state_cell = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

batch_size = 50
epochs = 50
# Building the training model:
training_model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
training_model.summary()
# Compile the model:
training_model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
# Train the model:
training_model.fit([encoder_input_data, decoder_input_data], decoder_target_data, batch_size=batch_size, epochs=epochs, validation_split=0.2)
# Save the model
training_model.save('training_model.h5')

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, None, 18)]   0           []                               
                                                                                                  
 input_4 (InputLayer)           [(None, None, 27)]   0           []                               
                                                                                                  
 lstm_2 (LSTM)                  [(None, 256),        281600      ['input_3[0][0]']                
                                 (None, 256),                                                     
                                 (None, 256)]                                                     
                                                                                            

In [14]:
from tensorflow import keras
from keras.layers import Input, LSTM, Dense
from keras.models import Model, load_model
import numpy as np

training_model = load_model('training_model.h5')
#Because using a saved model:
encoder_inputs = training_model.input[0]
encoder_outputs, state_h_enc, state_c_enc = training_model.layers[2].output
encoder_states = [state_h_enc, state_c_enc]

In [15]:
# Building the encoder test model
encoder_model = Model(encoder_inputs, encoder_states)
latent_dim = 256

# Building the two decoder state input layers:
decoder_state_input_hidden = Input(shape=(latent_dim,))
decoder_state_input_cell = Input(shape=(latent_dim,))

# Put the state input layers into a list:
decoder_states_inputs = [decoder_state_input_hidden, decoder_state_input_cell]

# Call the decoder LSTM:
decoder_outputs, state_hidden, state_cell = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_hidden, state_cell]
# Redefine the decoder outputs:
decoder_outputs = decoder_dense(decoder_outputs)
# Build the decoder test model:
decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)

In [16]:
def decode_sequence(test_input):
  # Encode the input as state vectors:
    encoder_states_value = encoder_model.predict(test_input)
  # Set decoder states equal to encoder final states
    decoder_states_value = encoder_states_value
  # Generate empty target sequence of length 1:
    target_seq = np.zeros((1, 1, num_decoder_tokens))
  # Populate the first token of target sequence with the start token:
    target_seq[0, 0, target_features_dict['<START>']] = 1.
  
    decoded_sentence = ''

  
    stop_condition = False
    while not stop_condition:
    # Run the decoder model to get possible output tokens (with probabilities) & states
        output_tokens, new_decoder_hidden_state, new_decoder_cell_state = decoder_model.predict([target_seq] + decoder_states_value)

    # Choose token with highest probability
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_token = reverse_target_features_dict[sampled_token_index]
        decoded_sentence += " " + sampled_token

    # Exit condition: either hit max length or find stop token.
        if (sampled_token == '<END>' or len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

    # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

    # Update states
        decoder_states_value = [new_decoder_hidden_state, new_decoder_cell_state]

    return decoded_sentence

In [17]:
for seq_index in range(11):
    test_input = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(test_input)
    print('-')
    print('Input sentence:', input_docs[seq_index])
    print('Decoded sentence:', decoded_sentence)

1/1 [==============================] - 0s 18ms/step
-
Input sentence: We'll see.
Decoded sentence:  veremos veremos
1/1 [==============================] - 0s 16ms/step
-
Input sentence: We'll see.
Decoded sentence:  veremos veremos
1/1 [==============================] - 0s 15ms/step
-
Input sentence: We'll try.
Decoded sentence:  Lo veremos .
1/1 [==============================] - 0s 15ms/step
-
Input sentence: We've won!
Decoded sentence:  Hemos ganado
1/1 [==============================] - 0s 16ms/step
-
Input sentence: Well done.
Decoded sentence:  Ya veremos .
1/1 [==============================] - 0s 15ms/step
-
Input sentence: What's up?
Decoded sentence:  ¿ ¿ ? ? <END>
1/1 [==============================] - 0s 16ms/step
-
Input sentence: Who cares?
Decoded sentence:  ¿ ¿ quién ? ?
1/1 [==============================] - 0s 15ms/step
-
Input sentence: Who drove?
Decoded sentence:  ¿ ¿ ? ? <END>
1/1 [==============================] - 0s 16ms/step
-
Input sentence: Who drove?
Decode